# Generating metadata

### What this notebook does
**Step 1:** Create metadata file, containing a bunch of JSON-formatted trial metadata objects

**Step 2:**  Insert each trial as a record into a mongo database

This assumes that the stimuli have been uploaded to the S3 bucket using `upload_stims_to_s3.ipynb`.

In [126]:
#Which experiment? bucket_name is the name of the experiment and will be name of the databases both on mongoDB and S3
bucket_name = 'human-physics-benchmarking-containment-pilot'
stim_version = 'example'

In [100]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import json
import pymongo as pm
from glob import glob
from IPython.display import clear_output
import ast
import itertools
import random
import h5py

In [101]:
def list_files(paths, ext='mp4'):
    """Pass list of folders if there are stimuli in multiple folders. 
    Make sure that the containing folder is informative, as the rest of the path is ignored in naming. 
    Also returns filenames as uploaded to S3"""
    if type(paths) is not list:
        paths = [paths]
    results = []
    names = []
    for path in paths:
        results += [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
        names += [os.path.basename(os.path.dirname(y))+'_'+os.path.split(y)[1] for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
    hdf5s = [r.split("_img.")[0]+".hdf5" for r in results]
    return results,names,hdf5s

In [102]:
## either
## where are your stimulus images stored?
#Where on disk are the stimuli stored?
data_dirs = ['/Users/choldawa/Documents/Projects/tdw_physics/tdw_physics/target_controllers/stimuli/pilot-containment-bowl',
                '/Users/choldawa/Documents/Projects/tdw_physics/tdw_physics/target_controllers/stimuli/pilot-containment-torus',
                '/Users/choldawa/Documents/Projects/tdw_physics/tdw_physics/target_controllers/stimuli/pilot-containment-multi'
                
            ]

dataset_name = '{}_{}'.format(bucket_name, stim_version)
stimulus_extension = "mp4" #what's the file extension for the stims? Provide without dot

## get a list of paths to each one
full_stim_paths,filenames, full_hdf5_paths = list_files(data_dirs,stimulus_extension)
print('We have {} stimuli to evaluate.'.format(len(full_stim_paths)))

We have 25 stimuli to evaluate.


In [103]:
# ## or
# local_stem = # path to stimuli dir
# dirnames = [d.split('/')[-1] for d in glob('./generation/pilot-XXX/pilot_XXX*')] # path to folder containing relevant subfolders
# data_dirs = [local_stem + d for d in dirnames]

# dataset_name = '{}_{}'.format(bucket_name, stim_version)
# stimulus_extension = "mp4" #what's the file extension for the stims? Provide without dot

# ## get a list of paths to each one
# full_stim_paths,filenames, full_hdf5_paths = list_files(data_dirs,stimulus_extension)
# print('We have {} stimuli to evaluate.'.format(len(full_stim_paths)))

In [104]:
# full_hdf5_paths

We also want to have a number of familiarization trials. Put a couple of hand selected filenames here. The stims are expected in the S3 bucket with that filename. 

In [105]:
familiarization_stems = [
    'pilot-containment-familiarization'
]
familiarization_filenames = [(familiarization_stems[0] + ('_%04d_img.mp4' % d)) 
                             for d in range(1,5)]
rng = np.random.RandomState(seed=0)
familiarization_filenames = list(rng.permutation(familiarization_filenames))
familiarization_filenames

['pilot-containment-familiarization_0003_img.mp4',
 'pilot-containment-familiarization_0004_img.mp4',
 'pilot-containment-familiarization_0002_img.mp4',
 'pilot-containment-familiarization_0001_img.mp4']

In [106]:
## helper to build stim urls
def build_s3_url(filename, bucket_name):    
    return 'https://{}.s3.amazonaws.com/{}'.format(bucket_name, filename)

In [107]:
## basic metadata lists
stim_urls = [build_s3_url(p,bucket_name) for p in filenames]
stim_IDs = [name.split('.')[0] for name in filenames]
familiarization_stim_urls = [build_s3_url(p,bucket_name) for p in familiarization_filenames]
familiarization_stim_IDs = [name.split('.')[0] for name in familiarization_filenames]
len(familiarization_stim_urls)

4

In [108]:
familiarization_stim_urls

['https://human-physics-benchmarking-containment-pilot.s3.amazonaws.com/pilot-containment-familiarization_0003_img.mp4',
 'https://human-physics-benchmarking-containment-pilot.s3.amazonaws.com/pilot-containment-familiarization_0004_img.mp4',
 'https://human-physics-benchmarking-containment-pilot.s3.amazonaws.com/pilot-containment-familiarization_0002_img.mp4',
 'https://human-physics-benchmarking-containment-pilot.s3.amazonaws.com/pilot-containment-familiarization_0001_img.mp4']

In [109]:
## convert to pandas dataframe
M = pd.DataFrame([stim_IDs,stim_urls]).transpose()
M.columns = ['stim_ID', 'stim_url']

## drop the stims that have the same stem as the familiarization trials
for fstem in familiarization_stems:
    M = M[~M['stim_ID'].str.contains(fstem)]

familiarization_M = pd.DataFrame([familiarization_stim_IDs,familiarization_stim_urls]).transpose()
familiarization_M.columns = ['stim_ID', 'stim_url']
# save the familiariaziation dict
familiarization_trials = familiarization_M.transpose().to_dict()
# needs to have strings as keys
familiarization_trials = {str(key):value for key, value in familiarization_trials.items()}

In [110]:
len(familiarization_M)

4

In [111]:
# remove some bad stimuli -- regenerate these
bad_stimuli = [
    #filename of bad stimuli
]

for nm in bad_stimuli:
    M = M[~M['stim_ID'].str.contains(nm)]

In [112]:
len(M)

25

Add metadata to the stimuli

From `metadata.json`:

In [113]:
# if needed, add code to add additional columns
# Add trial labels to the metadata using the stimulus metadata.json
target_hit_zone_labels = dict()
for _dir in full_data:
    with open(_dir + '/metadata.json', 'rb') as f:
        trial_metas = json.load(f)
        
#     for i,meta in enumerate(trial_metas):
#         stim_name = meta['stimulus_name']
#         if stim_name[:2] == "b\'":
#             stim_name = stim_name[2:-1]
#         if '/' in stim_name:
#             stim_name = '_'.join(stim_name.split('/'))
#         stim_name = '_'.join([stim_name, 'img'])
#         label = meta['does_target_contact_zone']
#         target_hit_zone_labels[stim_name] = label
    for i,meta in enumerate(trial_metas):
        stim_name = meta['stimulus_name']
        if stim_name == 'None': #recreate stimname from order in metadata
            stim_name = str(i).zfill(4)
            stim_name = _dir.split('/')[-1] + '_' + stim_name
        if stim_name[-4:] != "_img": stim_name+='_img' #stimnames need to end in "_img"
        label = meta['does_target_contact_zone']
        target_hit_zone_labels[stim_name] = label
             
print("num positive labels: %d" % sum(list(target_hit_zone_labels.values())))
print("num negative labels: %d" % (len(target_hit_zone_labels) - sum(list(target_hit_zone_labels.values()))))

GT = pd.DataFrame([list(target_hit_zone_labels.keys()), list(target_hit_zone_labels.values())]).transpose()
GT.columns = ['stim_ID', 'target_hit_zone_label']
if all([col not in M.columns for col in GT.columns[1:]]):
    M = M.merge(GT, on='stim_ID')
    print("added labels %s" % list(GT.columns[1:]))
    
if all([col not in familiarization_M.columns for col in GT.columns[1:]]):
    fam_keys = [k for k in list(target_hit_zone_labels.keys()) if (k+'.mp4') in familiarization_filenames]
    fam_labels = [target_hit_zone_labels[k] for k in fam_keys]
    familiarization_GT = pd.DataFrame([fam_keys, fam_labels]).transpose()
    familiarization_GT.columns = ['stim_ID', 'target_hit_zone_label']
    familiarization_M = familiarization_M.merge(familiarization_GT, on='stim_ID')

num positive labels: 16
num negative labels: 17
added labels ['target_hit_zone_label']


From `hdf5s` get all the metadata:

In [114]:
metadata = {} #holds all the metadata for all stimuli

for name,hdf5_path in zip([f.split('.')[0] for f in filenames],full_hdf5_paths):
    #load hdf5
    hdf5 = h5py.File(hdf5_path,'r') #get the static part of the HDF5
    stim_name = str(np.array(hdf5['static']['stimulus_name']))
    metadatum = {} #metadata for the current stimulus
    for key in hdf5['static'].keys():
        datum = np.array(hdf5['static'][key])
        if datum.shape == (): datum = datum.item() #unwrap non-arrays
        metadatum[key] = datum
    #close file
    hdf5.close()
    metadata[name] = metadatum

In [27]:
# metadata[list(metadata.keys())[0]]

Insert those metadatas into M:

In [115]:
for index in M.index:
    stim_name = M.at[index,'stim_ID']
    for key,value in metadata[stim_name].items():
        M.at[index,key] = str(value) #insert every item as string

Ok, so we now have a list of all the stimuli to evaluate. Let's create random orders of them and submit them to the database. Set the number of stimuli shown to one participant and the number of different orders to generate (`num_of_permutations`). Each of these sets can be shown to a participant, with the least often shown set being shown (so we can get away with fewer sets than subjects). 

In [116]:
len(M)

25

In [117]:
stims_per_run = len(M) #len(M) to show all stimuli
num_of_sets = 10

In [118]:
#generate list of stimuli as dicts
L = M.to_dict(orient='records')

In [119]:
#generate list of lists of shuffled, sampled w/o replacement stimuli
stim_sets = []
for i in range(num_of_sets):
    stim_set = random.sample(L, stims_per_run)
    stim_sets += [{str(num):stim for num,stim in enumerate(stim_set)}]

In [120]:
len(stim_sets)

10

In [121]:
familiarization_M

,stim_ID,stim_url,target_hit_zone_label
0,pilot-containment-familiarization_0003_img,https://human-physics-benchmarking-containment...,False
1,pilot-containment-familiarization_0004_img,https://human-physics-benchmarking-containment...,True
2,pilot-containment-familiarization_0002_img,https://human-physics-benchmarking-containment...,True
3,pilot-containment-familiarization_0001_img,https://human-physics-benchmarking-containment...,False


In [122]:
#these will be our familiarization trials
familiarization_trials = familiarization_M.transpose().to_dict()
familiarization_trials = {str(key):value for key, value in familiarization_trials.items()}
familiarization_trials

{'0': {'stim_ID': 'pilot-containment-familiarization_0003_img',
  'stim_url': 'https://human-physics-benchmarking-containment-pilot.s3.amazonaws.com/pilot-containment-familiarization_0003_img.mp4',
  'target_hit_zone_label': False},
 '1': {'stim_ID': 'pilot-containment-familiarization_0004_img',
  'stim_url': 'https://human-physics-benchmarking-containment-pilot.s3.amazonaws.com/pilot-containment-familiarization_0004_img.mp4',
  'target_hit_zone_label': True},
 '2': {'stim_ID': 'pilot-containment-familiarization_0002_img',
  'stim_url': 'https://human-physics-benchmarking-containment-pilot.s3.amazonaws.com/pilot-containment-familiarization_0002_img.mp4',
  'target_hit_zone_label': True},
 '3': {'stim_ID': 'pilot-containment-familiarization_0001_img',
  'stim_url': 'https://human-physics-benchmarking-containment-pilot.s3.amazonaws.com/pilot-containment-familiarization_0001_img.mp4',
  'target_hit_zone_label': False}}

Now we have a list of sequences of stimuli.

Set up ssh bridge to write to mongodb. Insert your username. If you don't have an SSH secret set yet, run `ssh -fNL 27017:127.0.0.1:27017 USERNAME@cogtoolslab.org` in your shell.

In [18]:
!ssh -fNL 27017:127.0.0.1:27017 choldawa@cogtoolslab.org

choldawa@cogtoolslab.org's password: 


In [127]:
# set vars 
auth = pd.read_csv('../auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user. Place it in the toplevel of the repo
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## cogtoolslab ip address

conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[dataset_name]

In [128]:
#⚠️drop collection if necessary. 
db.drop_collection(dataset_name) 

{'ok': 0.0,
 'errmsg': 'ns not found',
 'code': 26,
 'codeName': 'NamespaceNotFound'}

In [134]:
#get list of current collections
sorted(db.list_collection_names())

['block-construction-silhouette-exp01',
 'block-construction-silhouette-exp02',
 'causaldraw',
 'causaldraw_annotations',
 'causaldraw_annotations_patching',
 'causaldraw_identification',
 'causaldraw_intervention',
 'causaldraw_intervention_patching',
 'collabdraw_collab8_recog',
 'compositional-abstractions-prior-elicitation_example',
 'curiotower-tdw',
 'curiotower-tdw-height3Jitter3',
 'curiotower_curiodrop',
 'dominoes-pilot_example',
 'graphical_conventions_object_annotation',
 'graphical_conventions_semantic_mapping',
 'graphical_conventions_semantic_mapping_patching',
 'graphical_conventions_semantic_mapping_spline_version_old',
 'human-physics-benchmarking-clothiness-pilot_iteration_1',
 'human-physics-benchmarking-collision-pilot_iteration_1',
 'human-physics-benchmarking-containment-pilot_example',
 'human-physics-benchmarking-dominoes-pilot_example',
 'human-physics-benchmarking-drop-pilot_iteration_1',
 'human-physics-benchmarking-gravity-pilot_example',
 'human-physics-be

Let's **do it**!

In [130]:
## actually add data now to the database
for (i,m) in enumerate(stim_sets):
    coll.insert_one({'stims':m, 'familiarization_stims': familiarization_trials})
    print('{} of {}'.format(i+1, len(stim_sets)))
    clear_output(wait=True)

print('Done inserting records into mongo! The collection name is',dataset_name)

Done inserting records into mongo! The collection name is human-physics-benchmarking-containment-pilot_example


In [131]:
coll.estimated_document_count()

10

In [132]:
coll.find_one()

{'_id': ObjectId('607f52fc1a57e4cdfa3f37d2'),
 'stims': {'0': {'stim_ID': 'pilot-containment-multi_0002_img',
   'stim_url': 'https://human-physics-benchmarking-containment-pilot.s3.amazonaws.com/pilot-containment-multi_0002_img.mp4',
   'target_hit_zone_label': False,
   'attachent_type': "b'bowl'",
   'attachment_fixed': 'False',
   'attachment_id': '5',
   'base_id': '4',
   'base_type': "b'bowl'",
   'bounciness': '[0.         0.         0.         0.         0.68077418]',
   'color': '[[1.         1.         0.        ]\n [0.         1.         1.        ]\n [0.8        0.8        0.8       ]\n [0.8        0.8        0.8       ]\n [0.61037083 0.95574383 0.41710907]]',
   'distractors': '[]',
   'dynamic_friction': '[0.1        0.01       0.5        0.5        0.08830407]',
   'git_commit': "b'52db7987dafd511b4dc54b6bad48f70fe95d1cd1'",
   'initial_position': '[[ 0.          0.          0.        ]\n [-1.          0.          0.        ]\n [ 0.          0.          0.        ]\n [ 

In [133]:
list(coll.find())

[{'_id': ObjectId('607f52fc1a57e4cdfa3f37d2'),
  'stims': {'0': {'stim_ID': 'pilot-containment-multi_0002_img',
    'stim_url': 'https://human-physics-benchmarking-containment-pilot.s3.amazonaws.com/pilot-containment-multi_0002_img.mp4',
    'target_hit_zone_label': False,
    'attachent_type': "b'bowl'",
    'attachment_fixed': 'False',
    'attachment_id': '5',
    'base_id': '4',
    'base_type': "b'bowl'",
    'bounciness': '[0.         0.         0.         0.         0.68077418]',
    'color': '[[1.         1.         0.        ]\n [0.         1.         1.        ]\n [0.8        0.8        0.8       ]\n [0.8        0.8        0.8       ]\n [0.61037083 0.95574383 0.41710907]]',
    'distractors': '[]',
    'dynamic_friction': '[0.1        0.01       0.5        0.5        0.08830407]',
    'git_commit': "b'52db7987dafd511b4dc54b6bad48f70fe95d1cd1'",
    'initial_position': '[[ 0.          0.          0.        ]\n [-1.          0.          0.        ]\n [ 0.          0.          0